In [1]:
import signac
from polybinderCG.coarse_grain import System
import os
import gsd.hoomd

In [2]:
project = signac.get_project("../ua-target-runs/rg-re-pl-sweep/")

In [3]:
for job in project.find_jobs(
    {"remove_hydrogens": True}, {"done": True}
):
    if job.isfile("components.gsd"):
        continue
    try:
        sys = System(
            compound="PEKK",
            gsd_file=job.fn("sim_traj.gsd")
        )
    except:
        print(job.id)
        continue
    
    #for mol in sys.molecules:
        #mol.sequence = job.doc.chain_sequences[0]
        #mol.assign_types()
    
    for mon in sys.monomers():
        mon.generate_components(index_mapping="ring_plus_linkage_UA")
            
    sys.coarse_grain_trajectory(
        file_path = os.path.join(job.ws, "components.gsd"),
        first_frame=0,
        last_frame=-1,
        use_monomers=False,
        use_components=True
    )

/home/chris/miniconda3/envs/msibi/lib/python3.9/site-packages/signac/contrib/project.py:1075: FutureWarning: The doc_filter argument was deprecated in version 1.7 and will be removed in version 3.0. Users should instead use a filter with a 'doc.' prefix. For example, `doc_filter={'foo': 'bar'}` is equivalent to `filter={'doc.foo': 'bar'}`. See https://docs.signac.io/en/latest/query.html#query-namespaces for more information.
  warnings.warn(DOC_FILTER_WARNING, FutureWarning)
/tmp/ipykernel_92282/3141390872.py:23: DeprecatedWarning: ws is deprecated as of 1.8 and will be removed in 2.0. Use Job.path instead.
  file_path = os.path.join(job.ws, "components.gsd"),


# Create GSDs for MSDs

- Rename center chunk of particles, use those in MSD calcs

In [ ]:
project = signac.get_project("../../coarse-grain-sims/tm_ti_sweep/nvt-runs/")

In [ ]:
for job in project.find_jobs():
    if job.isfile("cg_msd_traj.gsd"):
        continue
    sys = System(gsd_file=job.fn("sim_traj.gsd"), atoms_per_monomer=1)
    for mol in sys.molecules:
        mol.sequence = "EKK"
        mol.assign_types()
    sys.rename_center_chunk(chunk_size=15, use_monomers=True)
    sys.coarse_grain_trajectory(
        file_path = os.path.join(job.ws, "cg_msd_traj.gsd"),
        first_frame=0,
        last_frame=-1,
        use_monomers=True
    )

# Copy the CG'd gsd files to descriptive file names in the workspace dir:

In [ ]:
import shutil

for job in project.find_jobs(filter={"remove_hydrogens": True}):
    fname = f"{job.sp.para_weight}pm.gsd"
    shutil.copyfile(
        job.fn("components.gsd"), os.path.join(project.workspace(),"..",fname)
    )

In [ ]:
print(project.workspace())